In [1]:
pip install pyspark

     -------------------------------------- 281.4/281.4 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 199.7/199.7 kB ? eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824024 sha256=c6a6efe5e7bcea8ef4d9c2256fda8bf7f52688a7d96651fbc3c984d7d905ab1b
  Stored in directory: c:\users\usuario\appdata\local\pip\cache\wheels\ad\fa\e3\0ed85d0d18ebd9e1344ffd6c1d1d0031af21eee1988ba36050
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession

In [3]:
# Crea un objeto SparkSession
spark = SparkSession.builder.appName("MiApp").getOrCreate()

In [4]:
# Lee los datos de un archivo CSV en un DataFrame
df = spark.read.format("csv").option("header", "true").load("House_Price_Regression.csv")

In [5]:
# Muestra los primeros 20 registros del DataFrame
df.show(20)

+-------------------+------------------+------------------+-------+--------------+---------------+--------------+----------------+------------------+--------------+--------------------+-----------------+--------------------------+---------------------+----------------+----------------------+-------------------------+-------------------+-------------------+---------------------+------------------+----------------------+-------------------+---------------+-----------------+------------------+-----------------+--------------------+-----------+
|neighborhood_recode|          latitude|         longitude|  price|no_of_bedrooms|no_of_bathrooms|quality_recode|maid_room_recode|unfurnished_recode|balcony_recode|barbecue_area_recode|central_ac_recode|childrens_play_area_recode|childrens_pool_recode|concierge_recode|covered_parking_recode|kitchen_appliances_recode|maid_service_recode|pets_allowed_recode|private_garden_recode|private_gym_recode|private_jacuzzi_recode|private_pool_recode|security_re